In [1]:
import torch
import itertools
from data.pathways.reactome import Reactome, ReactomeNetwork
import pandas as pd
import numpy as np


def get_map_from_layer(layer_dict):
  """
  :param: layer_dict: dictionary of connections (e.g {'pathway1': ['g1', 'g2', 'g3']}
  :return: dataframe map of layer (index = genes, columns = pathways, values = 1 if connected; 0 else)
  """
  pathways = layer_dict.keys()
  genes = list(itertools.chain.from_iterable(layer_dict.values()))
  genes = list(np.unique(genes))
  df = pd.DataFrame(index=pathways, columns=genes)
  for k, v in layer_dict.items():
    df.loc[k, v] = 1
  df = df.fillna(0)
  return df.T


def initLayers(layer):
  mapp = get_map_from_layer(layer)
  return torch.Tensor(mapp.to_numpy())


class SpareLayer(torch.nn.Module):
  def __init__(self, layer):
    super().__init__()
    self.layer = layer
    self.weight = torch.nn.Parameter(initLayers(self.layer), requires_grad=False)

  def forward(self, Input):
    return Input @ self.weight

  def getWeight(self):
    return self.weight


net = ReactomeNetwork()
layers = net.get_layers(n_levels=3)
print(len(layers))
sparseLayers = []
for i, layer in enumerate(layers[::-1]):
  sparseLayers += [SpareLayer(layer)]

ModuleNotFoundError: No module named 'torch'

In [41]:
print(sparseLayers[1].getWeight())

Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [37]:
x = torch.randn(size=(1,488), requires_grad=True)

In [44]:
print(sparseLayers[1])

SpareLayer()


In [42]:
print(SpareLayer[1](x))

TypeError: 'type' object is not subscriptable

In [46]:
torch.sparse_coo

torch.sparse_coo